In [1]:
import os
import re
import numpy as np
import time

In [2]:
def ext_data(direccion):
    with open(str(direccion), 'r') as archivo:
        nomb = ""
        dim = 0
        veh = 0
        cap = 0
        autos = []
        x_coord = []
        y_coord = []
        dema = {}  
        coordenadas = False  
        demandas = False  

        for linea in archivo:

            if linea.startswith("NAME"):
                nomb = (linea.strip().split(":")[1]).strip() + ".vrp"

            if linea.startswith("COMMENT"):
                resultado = re.search(r'of trucks:\s*(\d+)', linea, re.IGNORECASE)
                if resultado:
                    veh = int(resultado.group(1))

            if linea.startswith("VEHIC"):
                veh = int(linea.strip().split(":")[1])
                
            if linea.startswith("DIMEN"):
                dim = int(linea.strip().split(":")[1])

            elif linea.startswith("CAPAC"):
                cap = int(linea.strip().split(":")[1])

            elif linea.startswith("NODE_COORD_SECTION"):
                coordenadas = True
                continue

            elif linea.startswith("DEMAND_SECTION"):
                coordenadas = False
                demandas = True
                continue

            elif linea.startswith("DEPOT_SECTION"):
                demandas = False

            elif coordenadas:
                coords = linea.split()
                if len(coords) >= 3:  
                    x_coord.append(float(coords[1]))
                    y_coord.append(float(coords[2]))

            elif demandas:
                demands = linea.split()
                if len(demands) >= 2:
                    nodo, dem = int(demands[0]), int(demands[1])
                    dema[nodo - 1] = dem  

    for i in range(veh):
        autos.append(i+1)
        
    cap_x_veh = {i: cap for i in range(1, veh + 1)}

    return nomb, dim, cap_x_veh, x_coord, y_coord, dema, autos


    

In [3]:
def OVRP_Greedy(vehiculos, clientes, distancia, q, Q):
    rutas = {}
    vehiculos_disponibles = list(vehiculos)

    while len(clientes) > 0 and vehiculos_disponibles:
        vehiculo = vehiculos_disponibles[0]
        capacidad_actual = Q[vehiculo]
        ruta_actual = [0]
        distancia_recorrida = 0

        while capacidad_actual > 0 and clientes:
            cliente_mas_cercano = min(clientes, key=lambda c: distancia[(ruta_actual[-1], c)])
            if q[cliente_mas_cercano] <= capacidad_actual:
                capacidad_actual -= q[cliente_mas_cercano]
                distancia_recorrida += distancia[(ruta_actual[-1], cliente_mas_cercano)]
                ruta_actual.append(cliente_mas_cercano)
                clientes.remove(cliente_mas_cercano)
            else:
                break

        rutas[vehiculo] = ruta_actual
        vehiculos_disponibles.pop(0)

    return rutas

In [4]:

def listar_contenido_directorio(ruta_dic):
    try:
        
        contenido = os.listdir(ruta_dic)
        for elemento in contenido:
            nomb, dim, cap_x_veh, x_coord, y_coord, dema, autos = ext_data((str(ruta_dic) + "/" + str(elemento)))
            inst = nomb

            inicio = time.time()

            n = dim

            clientes = [i for i in range(n) if i != 0]

            nodos = [0] + clientes

            arcos = [(i,j) for i in nodos for j in nodos if i != j]

            q = dema

            vehiculos = autos

            Q = cap_x_veh

            X = x_coord
            Y = y_coord

            distancia = {(i,j): np.hypot(X[i] - X[j], Y[i] - Y[j]) for i in nodos for j in nodos if i != j}
            
            solucion = OVRP_Greedy(vehiculos, clientes, distancia, q, Q)

            distancia_total = 0

            for vehiculo, ruta in solucion.items():
                distancia_recorrida_vehiculo = sum(distancia[(ruta[i], ruta[i+1])] for i in range(len(ruta)-1))
                
                distancia_total += distancia_recorrida_vehiculo

            fin = time.time()
            tiempo_transcurrido = fin - inicio
            print("---------------------------------")
            print("\n")
            print("El script tardó {:.5f} segundos en ejecutar".format(tiempo_transcurrido), "la instancia", inst)
            print("Distancia total recorrida:", distancia_total)
            print("\n")
    except PermissionError:
        print(f"No se tienen permisos para acceder al directorio {ruta_dic}")

ruta_directorio = os.getcwd()

for raiz, directorios, archivos in os.walk(ruta_directorio):
    for nombre_directorio in directorios:
        ruta_completa = os.path.join(raiz, nombre_directorio)
        listar_contenido_directorio(ruta_completa)


No se tienen permisos para acceder al directorio c:\Users\diego\OneDrive\Escritorio\Final_Tesis\Codigo\INSTANCES
---------------------------------


El script tardó 0.00399 segundos en ejecutar la instancia A-n45-k7.vrp
Distancia total recorrida: 828.2375204073699


---------------------------------


El script tardó 0.00300 segundos en ejecutar la instancia A-n46-k7.vrp
Distancia total recorrida: 730.4216113499642


---------------------------------


El script tardó 0.00300 segundos en ejecutar la instancia A-n48-k7.vrp
Distancia total recorrida: 824.8146785849657


---------------------------------


El script tardó 0.00401 segundos en ejecutar la instancia A-n53-k7.vrp
Distancia total recorrida: 742.0764984435515


---------------------------------


El script tardó 0.00501 segundos en ejecutar la instancia A-n54-k7.vrp
Distancia total recorrida: 801.2188538035274


---------------------------------


El script tardó 0.00751 segundos en ejecutar la instancia A-n55-k9.vrp
Distancia 